In [1]:
import os.path
from scipy import *
import numpy as np
from numpy import *
import scipy as sp
from numpy import linalg as LA
from scipy import linalg as LA2
import sympy as sympy
import sys as sys
import time
import matplotlib.pyplot as plt
import itertools as it
from IPython.core.display import HTML


sys.path.append('/Users/sashacurcic/SashasDirectory/ANAG/FV_MG/')
from Modules import BasicTools as BT
from Modules import WaveTools as WT
from Modules import PlotTools as PT
from Modules import FFTTools as FFTT
from Modules import OperatorTools as OT
from Modules import GridTransferTools as GTT
from Modules import TestTools as TT
from Modules import SolverTools as ST
from Modules import WaveformTools as WFT

display(HTML("<style>pre { white-space: pre !important; }</style>"))
np.set_printoptions( linewidth = 10000, threshold = 100000)

In [2]:
nh_min = 16
refRatio = 2

deriv = 'C'
order = 4

L = 1
locs = [0.5]
epsilons = [1., 0.2] # [0.5, 0.5] # [1, 1]
mus = [1., 0.2] # [0.5, 0.5] # [1, 1]

In [3]:
omega = BT.Grid(nh_min)
finehalf = list(np.arange(int(nh_min / refRatio))) # list(np.arange(int((3 * nh_min) / (2 * refRatio))) + int(nh_min / (2 * refRatio)))
omega.AddPatch(refRatio, finehalf)
degFreed = omega.degFreed
nh_max = omega.nh_max
x = omega.xNode
xCell = omega.xCell

physics = BT.PhysProps(omega, epsilons, mus, locs, L)
matInd = physics.matInd


In [4]:
# matInd:      h
# offDiagInd:  k
# centCellInd: h + m + k + 1 for k < 0, h - m + k + 1 for k > 0

In [5]:
def CentGhostMaterial(omega, order, matInd, centCellInd, offDiagInd):
    errorLoc = 'ERROR:\nGridTransferTools:\nCenterGhostCellMaterial:\n'
    errorMess = ''
    
    degFreed = omega.degFreed
    
    stenc = np.zeros(degFreed, float)
    
    ghostStenc = GhostCellMaterialStencil(omega, order, matInd, centCellInd, offDiagInd)
    
    if ((type(offDiagInd) != int) or (offDiagInd == 0)):
        errorMess = 'offDiagInd must be a nonzero integer identifying the number of cells from material boundary!'
    else:
        if (offDiagInd < 0):
            stenc[matInd + 1:matInd + order + 2] = np.ones(order + 1) # ghostStenc
        else:
            stenc[matInd - order:matInd + 1] = np.ones(order + 1) # ghostStenc
    if (errorMess != ''):
        sys.exit(errorLoc + errorMess)
    
    return stenc

In [6]:
def GhostCellMaterialStencil(omega, order, matInd, centCellInd, offDiagInd):
    errorLoc = 'ERROR:\nGridTransferTools:\nGhostCellMaterialStencil:\n'
    errorMess = ''
    
    bounds = MaterialInterpBounds(omega, order, matInd, offDiagInd)

    xVec = MaterialInterpVec(omega, order, centCellInd, offDiagInd)

    polyInterp = GTT.MomentVander(order, bounds, xVec)
    
    return polyInterp

In [7]:
def MaterialInterpVec(omega, order, centCellInd, offDiagInd):
    xNode = omega.xNode
    xCell = omega.xCell
    
    intCoefs = (np.arange(order + 1) + 1)[::-1]**-1.
    polyCoefs = np.diag(intCoefs)
    
    matInd = physics.matInd
    
    bounds = xNode[centCellInd:centCellInd + 2]
    h = bounds[-1] - bounds[0]
    bounds = bounds - (offDiagInd * h)
    
    xValsR = np.polynomial.polynomial.polyvander(bounds[-1], order + 1)[0][1:][::-1] / h  
    xValsL = np.polynomial.polynomial.polyvander(bounds[0], order + 1)[0][1:][::-1] / h
    
    xVec = (xValsR - xValsL) @ polyCoefs
    
    return xVec

In [8]:
def MaterialInterpBounds(omega, order, matInd, offDiagInd):
    errorLoc = 'ERROR:\nGridTransferTools:\nMaterialInterpBounds:\n'
    errorMess = ''
    
    xNode = omega.xNode
    
    if ((type(offDiagInd) != int) or (offDiagInd == 0)):
        errorMess = 'offDiagInd must be a nonzero integer identifying the number of cells from material boundary!'
    else:
        if (offDiagInd > 0):
            print('this one.')
            bounds = xNode[matInd:matInd+order+2]
        else:
            bounds = xNode[matInd-order-1:matInd + 1]
    if (errorMess != ''):
        sys.exit(errorLoc + errorMess)
    
    print('Bounds are:', bounds, '<=', xNode[matInd])
    
    return bounds

In [9]:
print(x)
print('')
print(matInd)
a = CentGhostMaterial(omega, order, matInd, int(matInd - 1), -3)
print(a)
print('')

[0.      0.03125 0.0625  0.09375 0.125   0.15625 0.1875  0.21875 0.25    0.28125 0.3125  0.34375 0.375   0.40625 0.4375  0.46875 0.5     0.5625  0.625   0.6875  0.75    0.8125  0.875   0.9375  1.     ]

16
Bounds are: [0.34375 0.375   0.40625 0.4375  0.46875 0.5    ] <= 0.5
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0.]



In [10]:
def CDStencil(orderIn):
    if (orderIn % 2 == 0):
        order = orderIn
    else:
        order = int(orderIn + 1)
    
    loops = int(order / 2)
    print(loops)
        
    coefs = np.zeros(loops)
    stenc = np.zeros(order + 1)
    terms = np.arange(order + 1)
    rCell = np.asarray([1 / sp.math.factorial(j) for j in terms])
    lCell = rCell + 0
    lCell[1::2] = -lCell[1::2]
    deltaXFunc = lambda k: k ** terms
    tExp = [[] for j in range(loops)]
    for k in range(loops):
        rCellNew = rCell * deltaXFunc(k + 1)
        lCellNew = lCell * deltaXFunc(k + 1)
        tExp[k] = rCellNew - lCellNew


    tExp = np.asarray(tExp).transpose()
#     print('tExp:')
#     print(tExp)
#     print('')
    mat = tExp[1::2, :][1:, :-1]
#     print('mat:')
#     print(mat)
#     print('')

    vec = tExp[1::2, :][1:, -1]
    vec = -vec
#     print('vec:')
#     print(vec)
#     print('')
    coefs[-1] = 1
    coefs[:-1] = LA.inv(mat) @ vec
    stenc[:loops] = -coefs[::-1]
    stenc[loops + 1:] = coefs
    stenc = (-1)**(loops + 1) * stenc
    print(coefs)
    print(stenc)
#     print('coefs:')
#     print(coefs)
#     print('')
#     print('stenc:')
#     print(stenc)
#     print('')
    val = abs((tExp @ coefs)[1])
    stenc = stenc / val
#     print('val:')
#     print(val)
#     print('')
    
    return stenc

In [11]:
stenc = CDStencil(order)
print(stenc)
print('')

2
[-8.  1.]
[ 1. -8. -0.  8. -1.]
[ 0.08333333 -0.66666667 -0.          0.66666667 -0.08333333]



In [12]:
def UDStencil(orderIn):
    if (orderIn % 2 == 0):
        order = int(orderIn + 1)
    else:
        order = orderIn
    
    stenc = np.zeros(order + 1)
    
    if (order == 1):
        faceR = np.asarray([0, 1])
    else:
        if (order == 3):
            faceR = (1. / 6.) * np.asarray([0, -1, 5, 2])
        else:
            if (order == 5):
                faceR = (1. / 60.) * np.asarray([0, 2, -13, 47, 27, -3])
            else:
                if (order == 7):
                    faceR = (1. / 420.) * np.asarray([0, -3, 25, -101, 319, 214, -38, 4])
                else:
                    if (order == 9):
                        faceR = (1. / 2520.) * np.asarray([0, 4, -41, 199, -641, 1879, 1375, -305, 55, -5])
                    else:
                        errorMess = 'This program is not designed to handle this order of accuracy for forward- and backward-difference operators.'
    faceL = np.roll(faceR, -1)
    print(faceR)
    print(faceL)
    stenc = faceR - faceL
    
    return stenc

In [13]:
stenc = UDStencil(order)
print(stenc)
print('')

[ 0.          0.03333333 -0.21666667  0.78333333  0.45       -0.05      ]
[ 0.03333333 -0.21666667  0.78333333  0.45       -0.05        0.        ]
[-0.03333333  0.25       -1.          0.33333333  0.5        -0.05      ]



In [14]:
def UDStencil1(order):
    coefs = np.zeros(order + 1)
    stenc = np.zeros(order + 1)
    terms = np.arange(order + 1)
    cell = np.asarray([1 / sp.math.factorial(j) for j in terms])
    deltaXFunc = lambda k: k ** terms
    tExp = [[] for j in range(order + 1)]
    for k in range(order + 1):
        cellNew = cell * deltaXFunc(k)
        tExp[k] = cellNew
    
    tExp = np.asarray(tExp).transpose()
    mat = np.zeros((order, order), float)
    vec = np.zeros(order, float)
    mat[0, :] = tExp[0, :order]
    mat[1:, :] = tExp[2:, :order]
    vec[0] = tExp[0, order]
    vec[1:] = tExp[2:, order]
    vec = -vec
    stenc[order] = 1
    stenc[:order] = LA.inv(mat) @ vec
    val = (tExp @ stenc)[1]
    
#     UNCOMMENT THIS LINE!!!!!
#     stenc = stenc / val



#     print(tExp)
#     print('')
#     print(mat)
#     print('')
#     print(vec)
#     print('')
#     print(val)
#     print('')
#     stenc[:order] = coefs[::-1]
#     stenc[order + 1:] = -coefs
    return stenc

In [15]:
def DDStencil(order):
    stenc = -UDStencil(order)[::-1]
    return stenc

In [16]:
def CentGhost(omega, order, x_0, diff):
    errorLoc = 'ERROR:\nGridTransferTools:\nCentGhost:\n'
    errorMess = ''
    
    degFreed = omega.degFreed
    hs = omega.h
    
    spots = np.roll(hs, -1) - hs
    p = np.where(spots > 0)[0][0]
    q = np.where(spots < 0)[0][0]
    
    h_c = max(hs)
    h_f = min(hs)
    
    n_c_m = list(hs).count(h_c)
    n_f_m = list(hs).count(h_f)
    
    ghostCell, n_c, n_f = GTT.GhostCellStencil(order, x_0)
    
    if (n_c > n_c_m):
        errorMess = 'This grid has too few coarse cells for the order of the polynomial interpolation!'
    if (n_f > n_f_m):
        errorMess = 'This grid has too few fine cells for the order of the polynomial interpolation!'
    
    cells = n_c + n_f
    
    # ADD ERROR CHECKER FOR THE NUMBER OF COURSE AND FINE CELLS ON GRID OF GIVEN SIZE!!!
    

    fullStenc = np.zeros(degFreed, float)
    
    if (x_0 > 0):
        for k in range(cells):
            index = (p - n_f + k + 1) % degFreed
            fullStenc[index] = ghostCell[k]
    else:
        if (x_0 < 0):
            for k in range(cells):
                index = (q - n_c + k + 1) % degFreed
                fullStenc[index] = ghostCell[k]
        else:
            errorMess = 'Argument x_0 cannot be 0!'
    
    
    if (errorMess != ''):
        sys.exit(errorLoc + errorMess)
    
    
    return fullStenc

In [17]:
ghostCent = CentGhost(omega, order, 0.5, deriv)
print(ghostCent)
print('')
print(omega.xPatches)
print('')

[ 0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.08571429 -0.41428571  0.78571429  0.57142857 -0.02857143  0.          0.          0.          0.          0.          0.        ]

[[array([0.    , 0.0625, 0.125 , 0.1875, 0.25  , 0.3125, 0.375 , 0.4375, 0.5   , 0.5625, 0.625 , 0.6875, 0.75  , 0.8125, 0.875 , 0.9375, 1.    ])], [array([0.     , 0.03125, 0.0625 , 0.09375, 0.125  , 0.15625, 0.1875 , 0.21875, 0.25   , 0.28125, 0.3125 , 0.34375, 0.375  , 0.40625, 0.4375 , 0.46875, 0.5    ])]]



In [18]:
# FIGURE OUT IF YOU CAN USE THESE ARGUMENTS!
def MakeSpaceDeriv(omega, order, diff, matIndVal = []):
    errorLoc = 'ERROR:\nOperatorTools:\nMakeSpaceDeriv:\n'
    errorMess = ''
    if (diff == 'C' or diff == 'CD'):
        stenc = CDStencil(order)
        if (order % 2 == 0):
            orderStenc = order
        else:
            orderStenc = int(order + 1)
        off = int(orderStenc / 2)
        loBound = -off / 2.
        hiBound = off / 2.
    else:
        orderStenc = order
        if (order % 2 == 0):
            orderStenc = int(order + 1)
        else:
            orderStenc = order
        off = ((orderStenc + 1) / 2)
        if (diff == 'U' or diff == 'UD'):
            stenc = UDStencil(order)
            loBound = -off / 2.
            hiBound = (off - 1.) / 2.
        else:
            if (diff == 'D' or diff == 'DD'):
                stenc = DDStencil(order)
                off = int(off - 1)
                loBound = -off / 2.
                hiBound = (off + 1.) / 2.
                print('loBound:', loBound)
                print('hiBound:', hiBound)
            else:
                errorMess = 'Invalid entry for variable diff. Must be \'C\', \'U\', \'D\' \'CD\', \'UD\', or \'DD\'.'
    if (errorMess != ''):
        sys.exit(errorLoc + errorMess)
    
    # HERE IS WHERE YOU MADE SOME EDITS.
    if (matIndVal is not []):
        sign = np.sign(matIndVal)
        matInd = abs(matIndVal)
    else:
        sign = 0
    print('MATIND IS', matInd)
    print('SIGN IS', sign)
    
    stenc = np.ones(orderStenc + 1)
    
    degFreed = omega.degFreed
    hs = omega.h
    
    spots = np.roll(hs, -1) - hs
    # Index before fine-coarse interface
    p = np.where(spots > 0)[0][0]
    # Index before coarse-fine interface
    q = np.where(spots < 0)[0][0]
    
    polyStencSet = [[] for i in range(orderStenc)]
    cellFaces = np.linspace(loBound, hiBound, num = orderStenc + 1)
    zeroLoc = np.where(cellFaces == 0)[0][0]
    cellFaces = np.delete(cellFaces, zeroLoc)
    
    print(cellFaces)
    print('')
    

    for i in range(orderStenc):
        polyStencSet[i] = GTT.CentGhost(omega, order, cellFaces[i])
        polyStencSet[i][polyStencSet[i] != 0] = -3 - (i * 0.5)
    
    polyStencSet = np.asarray(polyStencSet)
    # DELETE THE LINES BELOW!!!
    print('AFTER:')
    print(polyStencSet)
    print('')

    IMat = np.eye(degFreed, degFreed)
    
    # YOU'RE GONNA NEED THESE TO RESTRICT FOR HIGHER EVEN ORDERS, TOO.
    
    
    
    polyMatU = IMat + 0
    
    
    mat = np.zeros((degFreed, degFreed), float)
    derivOp = mat + 0
    
    for d in range(orderStenc + 1):
        s = int(off - d)
        print('')
        print('BEGIN ITERATION:')
        print('k = ' + str(s))
        
        derivMat = mat + 0
        np.fill_diagonal(derivMat, stenc[d])
        derivMat = np.roll(derivMat, s, axis = 0)
        
        polyMat = IMat + 0

        if (s > 0):
            j = int(off - s)
            pAt = p
            pLow = (p - 1) % degFreed
            pHi = (p + 1) % degFreed
            qAt = (q - s + 1) % degFreed #(q + 1) % degFreed
            for i in range (s):
                print('m = ' + str(i + 1))
                print('')
#                 print('j =', j)
                polyMat[pAt, :] = 0
                polyMat[pAt, pLow:pHi] = 0.5
                polyMat[qAt, :] = polyStencSet[j, :]
                if (sign > 0):
                    print('IT WORKED!')
                    polyMat[matInd - i, :] = CentGhostMaterial(omega, order, matInd, int(matInd - i + s), s)
                print(polyMat)
                print('')
                pAt = (pAt - 1) % degFreed
                pLow = (pLow - 2) % degFreed
                pHi = (pHi - 2) % degFreed
                qAt = (qAt + 1) % degFreed
                j = int(j + 1)
                
        if (s < 0):
            
            j = int(off) # - s - 1
            qAt = (q + 1) % degFreed
            qLow = (q + 1) % degFreed
            qHi = (q + 3) % degFreed
            pAt = (p + 1) % degFreed#p
            for i in range(abs(s)):
                print('m = ' + str(-i - 1))
                print('')
#                 print('j =', j)
#                 print('pAt is', pAt)
                polyMat[qAt, :] = 0
                polyMat[qAt, qLow:qHi] = 0.5
                polyMat[pAt, :] = polyStencSet[j, :]
                if (sign < 0):
                    polyMat[matInd - i, :] = CentGhostMaterial(omega, order, matInd, int(matInd - i + s), s)
                print(polyMat)
                print('')
                qAt = (qAt + 1) % degFreed
                qLow = (qLow + 2) % degFreed
                qHi = (qHi + 2) % degFreed
                pAt = (pAt + 1) % degFreed
                j = int(j + 1) # - 1
        
        
        matThis = derivMat @ polyMat
        print('A^(' + str(s) + '):')
        print(polyMat)
        print('')
        print('D^(' + str(s) + '):')
        print(matThis)
        print('')
        print('END ITERATION ' + str(s))
        print('')
        
        derivOp = derivOp + matThis
    
    print('D:')
    print(derivOp)
    
    hMat = OT.StepMatrix(omega)
    
    derivOp = hMat @ derivOp
        
    return derivOp

In [27]:
print(np.sign(matInd))
A = MakeSpaceDeriv(omega, order, deriv, matIndVal = -matInd)


1
2
[-8.  1.]
[ 1. -8. -0.  8. -1.]
MATIND IS 16
SIGN IS -1
[-1.  -0.5  0.5  1. ]

AFTER:
[[-3.  -3.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  -3.  -3.  -3. ]
 [-3.5 -3.5 -3.5  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  -3.5 -3.5]
 [ 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  -4.  -4.  -4.  -4.  -4.   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  -4.5 -4.5 -4.5 -4.5 -4.5  0.   0.   0.   0.   0. ]]


BEGIN ITERATION:
k = 2
m = 1

[[ 1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0. 

In [20]:
# A = MakeSpaceDeriv(omega, order, deriv)
# B = OT.SpaceDeriv1(omega, order, deriv)

In [21]:
def BoundVals(order, x_0, ):
    a = abs(int(x_0 / 0.5))
    cells = order + 1
    if (cells <= a):
        n_c = cells
    else:
        n_c = a
        cells_new = cells - a
        if ((cells_new + 1) % 3 == 0):
            n_c_add = int(np.floor((cells_new + 1) / 3) - 1)
        else:
            n_c_add = int(np.floor((cells_new + 1) / 3))
        n_c = n_c + n_c_add
    n_f = cells - n_c
    
    if (n_f != 0):
        bounds = np.linspace(-n_c, n_f / 2., num = (2 * n_c) + n_f + 1)
        rm = [(2 * k) + 1 for k in range(n_c)]
        bounds = np.delete(bounds, rm)
        if (x_0 > 0):
            bounds = -bounds[::-1]
    else:
        if (x_0 % 1 == 0):
            bounds = np.arange(n_c + 1) + (int(x_0) - int((n_c + 1) / 2))
        else:
            bounds = np.arange(n_c + 1) + (int(x_0) - int(n_c / 2))
    return bounds, n_c, n_f

In [22]:
# dist is the number of cells away from the material boundary, indexed at 1.
# maxPoint is the maximum distance of the right face from the fine-coarse boundary or the left face from the coarse-fine boundary.

def JumpBounds(omega, physics, order, x_0):
    errorLoc = 'ERROR:\nGridTransferTools:\nJumpBounds:\n'
    errorMess = ''
    
    # if (number of cells between material boundary and AMR boundary <= order + 1):
    # or (all cells in material are uniform):
        # bounds = simple uniform, with last cell touching boundary
    # else:
        # if (the cells touching the material boundary are fine):
            # bounds = simple non-uniform, with last cell touching boundary
        # else:
            # if (the cells touching material boundary are coarse)
                # maxPoint = distance between AMR boundary and material boundary
                # bounds = BoundVals(order, x_0, maxPoint)
    
#     if (dist > order):
#         errorMess = 'dist cannot be greater than order!'
    
    h = omega.h
    x = omega.xNode
    locs = physics.locs
    
    h_max = max(h)
    h = h / h_max
    
    jump = np.where(x <= locs[0])[0][-1]
    
    print(x)
    print(x[jump])
    print('')
    
    if (errorMess != ''):
        sys.exit(errorLoc + errorMess)
    
    
    return h

In [23]:
# order = 2
# dist = 1
# h = JumpBounds(omega, physics, order, dist)

# print(h)
# print('')

In [24]:
# print(A)
# print('')
# print(B)
# print('')

In [25]:
# A = np.zeros((4, 4), float)
# A[:, 0] = 1

# A[0, 1] = - 3. / 2.
# A[1, 1] = - 1. / 2.
# A[2, 1] = 1. / 2.
# A[3, 1] = 3. / 2.

# A[0, 2] = 7. / 3.
# A[1, 2] = 1. / 3.
# A[2, 2] = 1. / 3.
# A[3, 2] = 7. / 3.

# A[0, 3] = - 15. / 4.
# A[1, 3] = - 1. / 4.
# A[2, 3] = 1. / 4.
# A[3, 3] = 15. / 4.

# B = LA.inv(A)

In [26]:
# print(A)
# print('')
# print(B)
# print('')
# print(sum(B, axis = 0))